Resources about numpy:
https://numpy.org/doc/stable/user/quickstart.html

Resources about pandas:
https://pandas.pydata.org/docs/reference/frame.html


In [46]:
!pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
!pip3 install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
!pip install openpyxl


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
# Import numpy and pandas
import numpy as np
import pandas as pd

In [50]:
# TODO: Create function of code below called system_reader which takes the file path of the system data as input
# and returns the line and bus data.

In [51]:
# Replace 'file.xlsx' with your actual file path
file_path = 'system_basecase.xlsx'

# Replace 'Sheet1' with the name of the sheet you want to read
sheet_name1 = 'BusData'
sheet_name2 = 'LineData'


# Read the Excel file into a pandas DataFrame for the specified sheet
bus_data = pd.read_excel(file_path, sheet_name=sheet_name1)
line_data = pd.read_excel(file_path, sheet_name=sheet_name2)

# Display the contents of the DataFrame
print(bus_data)
print(line_data)

    Bus #  P MW  Q MVAr Type  P Gen  V Set
0       1   0.0     0.0    S      0  1.040
1       2  20.1    12.5    D      0  1.000
2       3  45.2    16.0    G     42  1.025
3       4  60.8    10.9    D      0  1.000
4       5   7.6    -1.6    D      0  1.000
5       6  13.5     8.5    D      0  1.000
6       7  33.5    11.6    D      0  1.000
7       8   9.0     5.8    D      0  1.000
8       9   4.3     2.1    G     23  1.020
9      10   5.2     1.6    G     27  1.020
10     11  13.5     5.8    D      0  1.000
11     12  14.9     5.0    G     33  1.030
    From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0      1   2       0.01938       0.05917        0.0528         95
1      1   5       0.05203       0.20304        0.0492        100
2      2   3       0.04699       0.19797        0.0438       9999
3      2   4       0.05811       0.17632        0.0340       9999
4      2   5       0.05695       0.07388        0.0346       9999
5      3   4       0.67010       0.17103     

In [52]:
# TODO: Create function of the code below called admittance_matrix

In [53]:
# Calculating admittance matrix
num_buses = len(bus_data)
Ybus = np.zeros((num_buses, num_buses), dtype=complex)

# Test admitance matrix
# print(Ybus)

# Calculate total admittance of lines connected to each bus and store on the main diagonal
for i in range(num_buses):
    # Debug code
    # print('iteration', i)
    
    # Select all lines connected to the current bus in question.
    bus_lines = line_data[(line_data['From'] == i + 1) | (line_data['To'] == i + 1)]
    print(bus_lines)
    # Sum all line admittances.
    total_admittance = sum(1 / (line['Rtotal, p.u.'] + 1j * line['Xtotal, p.u.']) for _, line in bus_lines.iterrows())
    Ybus[i, i] = total_admittance

# Test calculations
# print(Ybus)

# Calculate admittance between each pair of buses and fill the off-diagonal elements
for _, line in line_data.iterrows():
    from_bus = int(line['From']) - 1
    to_bus = int(line['To']) - 1
    
    admittance = 1 / (line['Rtotal, p.u.'] + 1j * line['Xtotal, p.u.'])
    
    Ybus[from_bus, to_bus] = -admittance
    Ybus[to_bus, from_bus] = -admittance  # Assuming symmetric matrix

# print("Admittance Matrix (Ybus):")
print(Ybus)

   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.01938       0.05917        0.0528         95
1     1   5       0.05203       0.20304        0.0492        100
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
0     1   2       0.01938       0.05917        0.0528         95
2     2   3       0.04699       0.19797        0.0438       9999
3     2   4       0.05811       0.17632        0.0340       9999
4     2   5       0.05695       0.07388        0.0346       9999
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
2     2   3       0.04699       0.19797        0.0438       9999
5     3   4       0.67010       0.17103        0.0128       9999
   From  To  Rtotal, p.u.  Xtotal, p.u.  Btotal, p.u.  Fmax, MVA
3     2   4       0.05811       0.17632        0.0340       9999
5     3   4       0.67010       0.17103        0.0128       9999
6     4   5       0.01335       0.04211        0.0391       9999
7     4   7       0.00000

In [54]:
# Create 12x1 Matrix to store given V values
num_buses = len(bus_data)
V_bus = np.zeros(num_buses)
# print(num_buses)
for i in range(len(bus_data)):
    V_bus[i] = bus_data.iloc[i]['V Set']
# print(V_bus)

In [55]:
# Create 12 x 1 Matrix to initialize bus angle values
bus_angles = np.zeros(len(bus_data))
# print(bus_angles)

In [56]:
# Create 12 x 1 Matrix to store given P_gen values
num_buses = len(bus_data)
P_gen = np.zeros(num_buses)
# print(num_buses)
for i in range(len(bus_data)):
    P_gen[i] = bus_data.iloc[i]['P Gen']
# print(P_gen)

In [60]:
# Assuming Ybus is the calculated admittance matrix and bus_data contains bus information
# Assuming angles is a 12x1 matrix containing voltage angles for each bus in radians

# Number of buses
num_buses = len(bus_data)

# Initialize arrays to store calculated P and Q
P_calc = np.zeros(num_buses)
Q_calc = np.zeros(num_buses)

# Calculate P and Q injections at each bus
for i in range(num_buses):
    # Initialize loads with the known values
    P_calc[i] = P_gen[i] - bus_data.iloc[i]['P MW']
    Q_calc[i] = bus_data.iloc[i]['Q MVAr']

    # Test code
    print("i term", i, P_calc)
    print("i term", i, Q_calc)
    
    for j in range(num_buses):
        P_calc[i] += V_bus[i] * np.sum(V_bus[j] * (np.real(Ybus[i, j])) * np.cos(np.angle(Ybus[i,j])) + np.imag(Ybus[i,j]) * np.sin(np.angle(Ybus[i,j])))
        Q_calc[i] += V_bus[i] * np.sum(V_bus[j] * (np.real(Ybus[i, j])) * np.sin(np.angle(Ybus[i,j])) - np.imag(Ybus[i,j]) * np.cos(np.angle(Ybus[i,j])))
        
        # Test code
        print("j term", j, P_calc)
        print("j term", j, Q_calc)

# Test code
print(P_calc)
print(Q_calc)

# Calculate mismatches
mismatch_P = bus_data['P MW'] - P_calc
mismatch_Q = bus_data['Q MVAr'] - Q_calc

# Display mismatches
print("Mismatch in Active Power (P):")
print(mismatch_P)

print("\nMismatch in Reactive Power (Q):")
print(mismatch_Q)

i term 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
i term 0 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
j term 0 [21.73331337  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
j term 0 [-0.24562954  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
j term 1 [38.43666939  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
j term 1 [-0.24562954  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
j term 2 [38.43666939  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
j term 2 [-0.24562954  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]
j term 3 [38.43666

In [58]:
# Number of buses
num_buses = len(bus_data)

# Initialize array to store calculated active power P_i at each bus
P_calc = np.zeros(num_buses)

# Calculate active power injections at each bus
for i in range(num_buses):
    # Initialize active power with the known value from the input data
    P_calc[i] = bus_data.iloc[i]['P MW']
    
    for j in range(num_buses):
        # Calculate active power injection using the power flow equation
        V_i = np.abs(bus_data.iloc[i]['V Set'])
        V_j = np.abs(bus_data.iloc[j]['V Set'])
        angle_diff = np.angle(Ybus[i, j])
        G_ij = np.real(Ybus[i, j])
        B_ij = np.imag(Ybus[i, j])
        
        P_calc[i] -= V_i * V_j * (G_ij * np.cos(angle_diff) + B_ij * np.sin(angle_diff))

# Display calculated active power at each bus
print("Calculated Active Power (P_i) at each bus:")
print(P_calc)

Calculated Active Power (P_i) at each bus:
[-44.18838513 -54.33666134  32.6590549   -1.01401899 -76.0000261
 -24.08122494   1.08003586 -22.81935813 -14.91065509  -8.84197638
 -12.03686814   2.50915556]


In [59]:
# # Number of buses
# num_buses = len(bus_data)

# # Set the slack bus index and its reference angle
# slack_bus = 0  # Assuming the first bus is the slack bus (index 0)
# reference_angle = 0.0  # Angle in radians for the slack bus

# # Initialize array to store calculated angles for each bus
# angles = np.zeros(num_buses)

# # Set the reference angle for the slack bus
# angles[slack_bus] = reference_angle

# # Tolerance for convergence
# tolerance = 1e-6  # Adjust as needed for convergence

# # Maximum number of iterations for convergence
# max_iterations = 100  # Adjust as needed

# # Perform iterative calculation for angles
# for _ in range(max_iterations):
#     max_diff = 0.0  # Initialize maximum angle difference for convergence check
    
#     for i in range(num_buses):
#         if i != slack_bus:
#             # Calculate angle difference using complex voltages and the admittance matrix
#             sum_term = 0.0
#             for j in range(num_buses):
#                 if i != j:
#                     sum_term += np.abs(bus_data.iloc[i]['V Set']) * np.abs(bus_data.iloc[j]['V Set']) * \
#                                 (np.real(Ybus[i, j]) * np.sin(angles[j] - angles[i]) -
#                                  np.imag(Ybus[i, j]) * np.cos(angles[j] - angles[i]))
            
#             new_angle = np.arctan(sum_term / np.abs(bus_data.iloc[i]['V Set'])**2)
            
#             # Calculate maximum difference for convergence check
#             max_diff = max(max_diff, abs(new_angle - angles[i]))
            
#             # Update angle for non-reference buses
#             angles[i] = new_angle
    
#     # Check for convergence
#     if max_diff < tolerance:
#         print("Converged in", _+1, "iterations.")
#         break

# Display calculated angles for each bus
# print("Calculated Angles for Each Bus (radians):")
# print(angles)